## SORT + YOLO V8

### Importing Libraries

In [ ]:
from ultralytics import YOLO
import cv2
import math
from sort import *
import os

### Python Code for Real-time Object Detection and Tracking with YOLO and SORT


In [ ]:

totalCount=0

class ObjectDetection():

    def __init__(self, capture, result):
        self.capture = capture
        self.result = result
        self.model = self.load_model()
        self.CLASS_NAMES_DICT = self.model.model.names
        self.prev_frame_ids = {}  # Initialize dictionary to store previous frame IDs

    def load_model(self):
        model = YOLO("yolo_weights/yolov8n.pt")
        model.fuse()

        return model

    def predict(self, img):
        results = self.model(img, stream=True)
        return results

    def plot_boxes(self, results, detections):
        for r in results:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                w, h = x2 - x1, y2 - y1

                # Classname
                cls = int(box.cls[0])
                currentClass = self.CLASS_NAMES_DICT[cls]

                # Confidence score
                conf = math.ceil(box.conf[0]*100)/100

                if conf > 0.5 and currentClass == 'car':
                    currentArray = np.array([x1, y1, x2, y2, conf])
                    detections = np.vstack((detections, currentArray))

        return detections

    def track_detect(self, img, detections, tracker, totalCount):
        resultTracker = tracker.update(detections)

        for res in resultTracker:
            x1, y1, x2, y2, id = res
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

            if id not in self.prev_frame_ids:
                self.prev_frame_ids[id] = (x1 + x2) // 2

            if id in self.prev_frame_ids:
                curr_center = (x1 + x2) // 2
                prev_center = self.prev_frame_ids[id]

                totalCount += 1

                self.prev_frame_ids[id] = curr_center

        cv2.putText(img, f'Total Cars: {totalCount}', (10, 30), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)

        return img


    def __call__(self):

        cap = cv2.VideoCapture(self.capture)
        assert cap.isOpened()

        result_path = os.path.join(self.result, 'results.avi')

        codec = cv2.VideoWriter_fourcc(*'XVID')
        vid_fps = int(cap.get(cv2.CAP_PROP_FPS))
        vid_width, vid_height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        out = cv2.VideoWriter(result_path, codec, vid_fps, (vid_width, vid_height))

        tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3,)

#         totalCount = 0

        if not os.path.exists(self.result):
            os.makedirs(self.result)
            print("Result folder created successfully")
        else:
            print("Result folder already exists")

        while True:

            _, img = cap.read()
            assert _

            detections = np.empty((0, 5))
            results = self.predict(img)
            detections = self.plot_boxes(results, detections)
            detect_frame = self.track_detect(img, detections, tracker, totalCount)

            out.write(detect_frame)
            cv2.imshow('Image', detect_frame)
            if cv2.waitKey(1) == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()


detector = ObjectDetection(capture='C:/Users/aakas/Desktop/VIDEO-2024-03-16-23-08-40.mp4', result='result')
detector()


YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs
Result folder already exists

0: 384x640 5 persons, 3 cars, 2 motorcycles, 67.5ms
Speed: 2.7ms preprocess, 67.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 2 cars, 4 motorcycles, 1 truck, 61.5ms
Speed: 2.0ms preprocess, 61.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 3 cars, 3 motorcycles, 1 truck, 60.8ms
Speed: 2.4ms preprocess, 60.8ms inference, 12.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 4 cars, 4 motorcycles, 1 truck, 66.2ms
Speed: 2.9ms preprocess, 66.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 5 cars, 2 motorcycles, 1 truck, 65.0ms
Speed: 4.9ms preprocess, 65.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 5 cars, 4 motorcycles, 1 truck, 77.9ms
Speed: 1.7ms preprocess, 77.9ms inference